In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib editdistance sentencepiece
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, LlamaTokenizer, LlamaForCausalLM, pipeline
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd

from utils import similar_tag

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll
CUDA SETUP: CUDA runtime path found: C:\Users\panta\anaconda3\envs\nlp\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll...


c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: C:\Users\panta\anaconda3\envs\nlp did not contain ['cudart64_110.dll', 'cudart64_120.dll'] as expected! Searching further paths...
  warn(msg)
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/mingw-w64/bin'), WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/usr/bin')}
  warn(msg)


In [2]:
device = "cuda"
peft_model_id = r"ooferdoodles/text2tags-opt-350m"
base_model = "facebook/opt-350m"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model)

model = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16)
model.config

OPTConfig {
  "_name_or_path": "facebook/opt-350m",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": false,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 4096,
  "hidden_size": 1024,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "prefix": "</s>",
  "quantization_config": {
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_8bit": true
  },
  "torch_dtype": "float16",
  "transformers_version": "4.29.0.dev0",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 512
}

In [3]:
data = load_dataset("json", data_files=r"dataset/test_data.json")
tag_dict = similar_tag.load_dict()
data

Found cached dataset json (C:/Users/panta/.cache/huggingface/datasets/json/default-5cff6cb24ab66f93/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'tag_string'],
        num_rows: 150
    })
})

In [4]:
generation_config = GenerationConfig(
    temperature=1,
    top_p=1,
    top_k=40,
    num_beams=4,
    typical_p=1,
    do_sample=True,
    max_new_tokens=300,
    use_cache=True,
    no_repeat_ngram_size=3,
    # pad_token_id=model.config.eos_token_id
    # truncation_length=2048,
    # min_length=0,
    # add_bos_token=True,
    # ban_eos_token=False,
    # skip_special_tokens=True,
    # stopping_strings=[],
    # penalty_alpha=0,
    # repetition_penalty=2.5,
    # encoder_repetition_penalty=1,
)

In [20]:
tokenizer("2boys")

{'input_ids': [2, 176, 22505], 'attention_mask': [1, 1, 1]}

In [21]:
tokenizer.decode([176, 22505])

'2boys'

In [11]:
def pipe(data_point):
    # max_new_tokens = int(len(data_point['caption_string']) * max_token_scale)
    prompt = f"### Caption: {data_point['caption_string']}\n### Tags: "
    tokenized_prompt = tokenizer(
        prompt, 
        return_tensors='pt', 
        add_special_tokens=True).to(device)

    with torch.no_grad():
        output_tokens = model.generate(
            input_ids=tokenized_prompt['input_ids'],
            generation_config=generation_config,
        )[0]
    preds = tokenizer.decode(output_tokens, skip_special_tokens=True)
    data_point['raw_preds'] = preds
    pred_list = [x.strip() for x in preds.split('### Tags:')[-1].split(",")]
    corrected_tags = similar_tag.correct_tags(pred_list, tag_dict)
    data_point['tags'] = data_point['tag_string'].split(', ')
    data_point['pred_tags'] = corrected_tags
    return data_point

In [6]:
processed_data = data.map(pipe)
processed_data

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
def evaluate_accuracy(data_point):
    correct_count = len(set(data_point['tags']).intersection(data_point['pred_tags']))
    # incorrect_count = len(data_point['tags']) - correct_count
    data_point['accuracy'] = correct_count / len(data_point['tags']) * 100
    return data_point

In [ ]:
evaluated_data = processed_data.map(evaluate_accuracy)
evaluated_data

In [ ]:
df = evaluated_data['train'].to_pandas()
df

In [ ]:
df['accuracy'].mean()

In [12]:
ogey = pipe(data['train'][0])

In [19]:
print(ogey['tags'])
print(ogey['raw_preds'])

['1girl', 'breasts', 'closed_mouth', 'gloves', 'horns', 'large_breasts', 'long_hair', 'looking_at_viewer', 'monochrome', 'simple_background', 'smile', 'solo', 'star_ocean', 'star_ocean_anamnesis', 'star_ocean_till_the_end_of_time', 'white_background', 'wings']
### Caption: With her mesmerizing gaze and ethereal wings, Clair Lasbard, from the Star Ocean franchise, comes to life in monochrome hues thanks to the exquisite artistry of kiikii_(kitsukedokoro). Her alluring smile and intricate horns, complemented by her long hair and gloves, make for a stunning portrayal of this character known for her large breasts. Amidst a simple white background, this piece captures the beauty and mystique of Clair, leaving us entranced by her presence.
### Tags: 〈, :d, :o, blonde_hair, black_legs, blue_eyes, breasts, gloves, breasts_clair_lasbard, hair_ornament, hairband, hairclip, hairornament_gloves, large_breasts, looking_at_viewer, multiple_girls, star_ocean_(series), white_background, white_
